# 1. 그레이디언트 소실과 폭주 문제

- 그레이디언트 손실 : 훈련이 좋은 솔루션으로 수렴되지 않는 것
- 그레이디언트 폭주 : 그레이디언트가 점점 커져서 여러 층이 비정상저으로 큰 가중치로 갱신되면 알고리즘 발산

### 1.2 수렵하지 않는 화성화 함수
- LeakyReLU()가 ReLU()보다 항상 성능이 높다는 것

In [5]:
from tensorflow import keras


(X_train_full, y_train_full), (X_test, y_test) = keras.datasets.fashion_mnist.load_data()
X_train_full = X_train_full / 255.0
X_test = X_test / 255.0
X_valid, X_train = X_train_full[:5000], X_train_full[5000:]
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]

In [6]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.LeakyReLU(),
    keras.layers.Dense(10, activation="softmax")
])

In [7]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [8]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 3s 2ms/step - loss: 1.3422 - accuracy: 0.5798 - val_loss: 0.9037 - val_accuracy: 0.7056
Epoch 2/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.8101 - accuracy: 0.7294 - val_loss: 0.7172 - val_accuracy: 0.7640
Epoch 3/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.6892 - accuracy: 0.7699 - val_loss: 0.6398 - val_accuracy: 0.7866
Epoch 4/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.6259 - accuracy: 0.7925 - val_loss: 0.5873 - val_accuracy: 0.8084
Epoch 5/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.5848 - accuracy: 0.8059 - val_loss: 0.5559 - val_accuracy: 0.8178
Epoch 6/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.5565 - accuracy: 0.8134 - val_loss: 0.5345 - val_accuracy: 0.8204
Epoch 7/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.5354 - accuracy: 0.8193 - val_loss: 0.5141 - val_accuracy:

---
- PReLU() : 대규모 이미지데이터셋에서는 ReLU()보다 성능이 크게 앞서지만, 소규모 데이터셋에서는 훈려세트에 과대적합될 위험이 있음

In [9]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, kernel_initializer="he_normal"),
    keras.layers.PReLU(),
    keras.layers.Dense(100, kernel_initializer="he_normal"),
    keras.layers.PReLU(),
    keras.layers.Dense(10, activation="softmax")
])

In [10]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [11]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 4s 2ms/step - loss: 1.4048 - accuracy: 0.5740 - val_loss: 0.9370 - val_accuracy: 0.7186
Epoch 2/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.8342 - accuracy: 0.7391 - val_loss: 0.7393 - val_accuracy: 0.7652
Epoch 3/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.7027 - accuracy: 0.7726 - val_loss: 0.6494 - val_accuracy: 0.7896
Epoch 4/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.6343 - accuracy: 0.7912 - val_loss: 0.6015 - val_accuracy: 0.7986
Epoch 5/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.5904 - accuracy: 0.8046 - val_loss: 0.5608 - val_accuracy: 0.8194
Epoch 6/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.5597 - accuracy: 0.8136 - val_loss: 0.5347 - val_accuracy: 0.8268
Epoch 7/10
1719/1719 [==============================] - 3s 2ms/step - loss: 0.5372 - accuracy: 0.8193 - val_loss: 0.5155 - val_accuracy:

### 1.3 배치 정규화
- 현재 모델에서는 은닉층이 두개여서 큰 도움이 되지 않으 수도 있지만 깊은 네트워크에서는 엄청난 차이를 만들 수 있다.
- 배치 정규화는 활성화 함수 이후보다 활성화 함수 이전에 배치 정규화 층을 추가하는 것이 좋다고 조언
- 하지만 데이터셋에 가장 잘 맞는지 확인하는 것이 좋음

In [12]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),             # 두 개는 훈련되고 두개는 훈련되지 않음!
    keras.layers.Dense(300, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="relu"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])
# 배치 정규화 층은 입력마다 네 개의 파라미니터를 추가 (784 x 4 = 3136)

In [13]:
model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_2 (Flatten)         (None, 784)               0         
                                                                 
 batch_normalization (BatchN  (None, 784)              3136      
 ormalization)                                                   
                                                                 
 dense_6 (Dense)             (None, 300)               235500    
                                                                 
 batch_normalization_1 (Batc  (None, 300)              1200      
 hNormalization)                                                 
                                                                 
 dense_7 (Dense)             (None, 100)               30100     
                                                                 
 batch_normalization_2 (Batc  (None, 100)             

In [14]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [15]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.8722 - accuracy: 0.7067 - val_loss: 0.5727 - val_accuracy: 0.8118
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5802 - accuracy: 0.7995 - val_loss: 0.4872 - val_accuracy: 0.8352
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5197 - accuracy: 0.8183 - val_loss: 0.4495 - val_accuracy: 0.8480
Epoch 4/10
1719/1719 [==============================] - 6s 3ms/step - loss: 0.4853 - accuracy: 0.8295 - val_loss: 0.4265 - val_accuracy: 0.8544
Epoch 5/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4572 - accuracy: 0.8406 - val_loss: 0.4103 - val_accuracy: 0.8596
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4426 - accuracy: 0.8446 - val_loss: 0.3972 - val_accuracy: 0.8634
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4261 - accuracy: 0.8504 - val_loss: 0.3870 - val_accuracy:

In [16]:
# 배치 정규화 활성화 함수 앞에 배치

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(100, use_bias=False),
    keras.layers.BatchNormalization(),
    keras.layers.Activation("relu"),
    keras.layers.Dense(10, activation="softmax")
])

In [17]:
model.compile(loss="sparse_categorical_crossentropy",
              optimizer=keras.optimizers.SGD(learning_rate=1e-3),
              metrics=["accuracy"])

In [18]:
history = model.fit(X_train, y_train, epochs=10,
                    validation_data=(X_valid, y_valid))

Epoch 1/10
1719/1719 [==============================] - 6s 3ms/step - loss: 1.0404 - accuracy: 0.6780 - val_loss: 0.6682 - val_accuracy: 0.7950
Epoch 2/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.6769 - accuracy: 0.7824 - val_loss: 0.5545 - val_accuracy: 0.8176
Epoch 3/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5925 - accuracy: 0.8047 - val_loss: 0.5008 - val_accuracy: 0.8316
Epoch 4/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5472 - accuracy: 0.8164 - val_loss: 0.4699 - val_accuracy: 0.8408
Epoch 5/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.5110 - accuracy: 0.8273 - val_loss: 0.4484 - val_accuracy: 0.8466
Epoch 6/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4931 - accuracy: 0.8316 - val_loss: 0.4306 - val_accuracy: 0.8532
Epoch 7/10
1719/1719 [==============================] - 5s 3ms/step - loss: 0.4745 - accuracy: 0.8374 - val_loss: 0.4191 - val_accuracy:

### 1.4 그레이디언트 클리핑
- 그레이디언트 폭주문제를 완화하기 인기 있는 다른 방법은 역전파되 때 일저 임계값을 넘어서지 못하게 그레이디언트를 잘라내는 것
- 옵티마이저를 만들 때 clipvalue와 clipnorm 매개변수를 지정

In [ ]:
optimizer = keras.optimizers.SGD(clipvalue=1.0)
optimizer = keras.optimizers.SGD(clipnorm=1.0)

# 2. 사전훈련된 층 재사용하기
- 신경망의 하위층을 재사용하는 것을 **전이 학습**이라고 함

In [21]:
# MNIST 

import numpy as np
def split_dataset(X, y):
    y_5_or_6 = (y == 5) | (y == 6) # sandals or shirts
    y_A = y[~y_5_or_6]
    y_A[y_A > 6] -= 2 # class indices 7, 8, 9 should be moved to 5, 6, 7
    y_B = (y[y_5_or_6] == 6).astype(np.float32) # binary classification task: is it a shirt (class 6)?
    return ((X[~y_5_or_6], y_A),
            (X[y_5_or_6], y_B))

(X_train_A, y_train_A), (X_train_B, y_train_B) = split_dataset(X_train, y_train)
(X_valid_A, y_valid_A), (X_valid_B, y_valid_B) = split_dataset(X_valid, y_valid)
(X_test_A, y_test_A), (X_test_B, y_test_B) = split_dataset(X_test, y_test)
X_train_B = X_train_B[:200]
y_train_B = y_train_B[:200]

In [23]:
X_train_A.shape

(43986, 28, 28)

In [24]:
X_train_B.shape

(200, 28, 28)

In [25]:
y_train_A[:30]

array([4, 0, 5, 7, 7, 7, 4, 4, 3, 4, 0, 1, 6, 3, 4, 3, 2, 6, 5, 3, 4, 5,
       1, 3, 4, 2, 0, 6, 7, 1], dtype=uint8)

In [26]:
y_train_B[:30]

array([1., 1., 0., 0., 0., 0., 1., 1., 1., 0., 0., 1., 1., 0., 0., 0., 0.,
       0., 0., 1., 1., 0., 0., 1., 1., 0., 1., 1., 1., 1.], dtype=float32)

In [27]:
model_A = keras.models.Sequential()

model_A.add(keras.layers.Flatten(input_shape=[28, 28])) # 입력층

for n_hidden in (300, 100, 50, 50, 50):
    model_A.add(keras.layers.Dense(n_hidden, activation="selu")) # 은닉층
    
model_A.add(keras.layers.Dense(8, activation="softmax")) # 츌력층

In [28]:
model_A.compile(loss="sparse_categorical_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [29]:
# 모델 학습
history = model_A.fit(X_train_A, y_train_A, epochs=20,
                    validation_data=(X_valid_A, y_valid_A))

Epoch 1/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.5929 - accuracy: 0.8032 - val_loss: 0.3944 - val_accuracy: 0.8617
Epoch 2/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.3571 - accuracy: 0.8747 - val_loss: 0.3268 - val_accuracy: 0.8869
Epoch 3/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.3204 - accuracy: 0.8871 - val_loss: 0.3024 - val_accuracy: 0.8949
Epoch 4/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.3006 - accuracy: 0.8948 - val_loss: 0.2922 - val_accuracy: 0.8966
Epoch 5/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.2869 - accuracy: 0.9004 - val_loss: 0.2805 - val_accuracy: 0.9028
Epoch 6/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.2763 - accuracy: 0.9045 - val_loss: 0.2695 - val_accuracy: 0.9083
Epoch 7/20
1375/1375 [==============================] - 3s 2ms/step - loss: 0.2685 - accuracy: 0.9086 - val_loss: 0.2634 - val_accuracy:

In [30]:
# 저정
model_A.save("my_model_A.h5")

In [31]:
model_B = keras.models.Sequential()
model_B.add(keras.layers.Flatten(input_shape=[28, 28]))
for n_hidden in (300, 100, 50, 50, 50):
    model_B.add(keras.layers.Dense(n_hidden, activation="selu"))
model_B.add(keras.layers.Dense(1, activation="sigmoid"))

In [32]:
model_B.compile(loss="binary_crossentropy",
                optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                metrics=["accuracy"])

In [33]:
history = model_B.fit(X_train_B, y_train_B, epochs=20,
                      validation_data=(X_valid_B, y_valid_B))

Epoch 1/20
7/7 [==============================] - 1s 33ms/step - loss: 0.5693 - accuracy: 0.7450 - val_loss: 0.5159 - val_accuracy: 0.8073
Epoch 2/20
7/7 [==============================] - 0s 11ms/step - loss: 0.4737 - accuracy: 0.8700 - val_loss: 0.4417 - val_accuracy: 0.8712
Epoch 3/20
7/7 [==============================] - 0s 11ms/step - loss: 0.4025 - accuracy: 0.9150 - val_loss: 0.3857 - val_accuracy: 0.9067
Epoch 4/20
7/7 [==============================] - 0s 11ms/step - loss: 0.3490 - accuracy: 0.9500 - val_loss: 0.3427 - val_accuracy: 0.9270
Epoch 5/20
7/7 [==============================] - 0s 11ms/step - loss: 0.3066 - accuracy: 0.9550 - val_loss: 0.3078 - val_accuracy: 0.9391
Epoch 6/20
7/7 [==============================] - 0s 11ms/step - loss: 0.2730 - accuracy: 0.9750 - val_loss: 0.2802 - val_accuracy: 0.9513
Epoch 7/20
7/7 [==============================] - 0s 12ms/step - loss: 0.2456 - accuracy: 0.9750 - val_loss: 0.2574 - val_accuracy: 0.9564
Epoch 8/20
7/7 [===========

In [34]:
model_B.summary()

Model: "sequential_5"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 flatten_5 (Flatten)         (None, 784)               0         
                                                                 
 dense_18 (Dense)            (None, 300)               235500    
                                                                 
 dense_19 (Dense)            (None, 100)               30100     
                                                                 
 dense_20 (Dense)            (None, 50)                5050      
                                                                 
 dense_21 (Dense)            (None, 50)                2550      
                                                                 
 dense_22 (Dense)            (None, 50)                2550      
                                                                 
 dense_23 (Dense)            (None, 1)                

In [35]:
########
# model_A와 model_B_on_A는 일부 층을 공유

model_A = keras.models.load_model("my_model_A.h5")
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

# model_B_on_A를 훈련할 때 model_A도 영향을 받는다 원치 않는다면 model_A를 클론해야 한다. 아래

In [36]:
# model_A 복제 후 학습

model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())
model_B_on_A = keras.models.Sequential(model_A_clone.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

In [37]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])

In [38]:
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=4,
                           validation_data=(X_valid_B, y_valid_B))

for layer in model_B_on_A.layers[:-1]:
    layer.trainable = True

model_B_on_A.compile(loss="binary_crossentropy",
                     optimizer=keras.optimizers.SGD(learning_rate=1e-3),
                     metrics=["accuracy"])
history = model_B_on_A.fit(X_train_B, y_train_B, epochs=16,
                           validation_data=(X_valid_B, y_valid_B))

Epoch 1/4
7/7 [==============================] - 1s 34ms/step - loss: 1.2807 - accuracy: 0.4400 - val_loss: 1.3019 - val_accuracy: 0.4604
Epoch 2/4
7/7 [==============================] - 0s 10ms/step - loss: 1.1781 - accuracy: 0.4600 - val_loss: 1.1969 - val_accuracy: 0.4787
Epoch 3/4
7/7 [==============================] - 0s 10ms/step - loss: 1.0768 - accuracy: 0.4700 - val_loss: 1.0989 - val_accuracy: 0.4949
Epoch 4/4
7/7 [==============================] - 0s 10ms/step - loss: 0.9829 - accuracy: 0.4850 - val_loss: 1.0085 - val_accuracy: 0.5051
Epoch 1/16
7/7 [==============================] - 1s 32ms/step - loss: 0.6520 - accuracy: 0.6200 - val_loss: 0.4745 - val_accuracy: 0.8022
Epoch 2/16
7/7 [==============================] - 0s 11ms/step - loss: 0.3548 - accuracy: 0.9050 - val_loss: 0.3403 - val_accuracy: 0.9057
Epoch 3/16
7/7 [==============================] - 0s 11ms/step - loss: 0.2609 - accuracy: 0.9800 - val_loss: 0.2687 - val_accuracy: 0.9402
Epoch 4/16
7/7 [===============

In [39]:
model_B.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 1ms/step - loss: 0.1324 - accuracy: 0.9760


[0.13239842653274536, 0.9760000109672546]

In [40]:
model_B_on_A.evaluate(X_test_B, y_test_B)

63/63 [==============================] - 0s 1ms/step - loss: 0.0827 - accuracy: 0.9905


[0.08267734944820404, 0.9904999732971191]

#### 14장에서 다시 학습
---

# 3.고속 옵티마이저

#### 3.1 모멘텀 옵티마이저

In [ ]:
optimizer = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9)

#### 3.2 네스테로프 가속 경사

In [ ]:
optimizer = keras.optimizers.SGD(learning_rate=0.001, momentum=0.9, nesterov=True)

#### 3.3 AdaGrad

In [ ]:
optimizer = keras.optimizers.Adagrad(learning_rate=0.001)

#### 3.4 RMSProp
- AdaGrad는 너무 빠릴 느려져서 저녁 최적점에 수렴하지 못하는 위험이 있음
- 아주 간단한 문제를 제외하고 이 옵티마이저가 언제나 AdaGrad 보다 훨신 더 성능이 좋음

In [ ]:
optimizer = keras.optimizers.RMSprop(learning_rate=0.001, rho=0.9)

#### 3.5 Adam과 Nadam 최적화
- 적응적 모멘트 추정(Adaptive moment estimation)을 의미하는 Adam은 모멘텀 최적화와 RMSProp의 아이디어를 합친 것

In [ ]:
# Adam
optimizer = keras.optimizers.Adamax(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

In [ ]:
# Nadam
optimizer = keras.optimizers.Nadam(learning_rate=0.001, beta_1=0.9, beta_2=0.999)

#### 3.6 학습률 스케쥴링

In [42]:
# 케라스에서 거듭제곱 기반 스케줄링이 가장 구현하기 쉽다. 옵티마이저를 만들 때 decay 매개변수만 지정하면 됨
optimizer = keras.optimizers.SGD(learning_rate=0.01, decay=1e-4)

In [46]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])

In [49]:
n_epochs = 25
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

NameError: name 'X_train_scaled' is not defined

# 4. 규제를 사용해 과대적합 피하기

#### 4.1 l1과 l2 규제

In [1]:
layer = keras.layers.Dense(100, activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))
# or l1(0.1) for ℓ1 regularization with a factor of 0.1
# or l1_l2(0.1, 0.01) for both ℓ1 and ℓ2 regularization, with factors 0.1 and 0.01 respectively

NameError: name 'keras' is not defined

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(300, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(100, activation="elu",
                       kernel_initializer="he_normal",
                       kernel_regularizer=keras.regularizers.l2(0.01)),
    keras.layers.Dense(10, activation="softmax",
                       kernel_regularizer=keras.regularizers.l2(0.01))
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

In [ ]:
from functools import partial

RegularizedDense = partial(keras.layers.Dense,
                           activation="elu",
                           kernel_initializer="he_normal",
                           kernel_regularizer=keras.regularizers.l2(0.01))

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

#### 4.2 드롭아웃

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

#### Alpha Dropout

In [ ]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax")
])
optimizer = keras.optimizers.SGD(learning_rate=0.01, momentum=0.9, nesterov=True)
model.compile(loss="sparse_categorical_crossentropy", optimizer=optimizer, metrics=["accuracy"])
n_epochs = 20
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))

In [ ]:
model.evaluate(X_test_scaled, y_test)


In [ ]:
model.evaluate(X_train_scaled, y_train)


In [ ]:
history = model.fit(X_train_scaled, y_train)


#### MC Dropout

In [ ]:
layer = keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal",
                           kernel_constraint=keras.constraints.max_norm(1.))

In [ ]:
MaxNormDense = partial(keras.layers.Dense,
                       activation="selu", kernel_initializer="lecun_normal",
                       kernel_constraint=keras.constraints.max_norm(1.))

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    MaxNormDense(300),
    MaxNormDense(100),
    keras.layers.Dense(10, activation="softmax")
])
model.compile(loss="sparse_categorical_crossentropy", optimizer="nadam", metrics=["accuracy"])
n_epochs = 2
history = model.fit(X_train_scaled, y_train, epochs=n_epochs,
                    validation_data=(X_valid_scaled, y_valid))